# Caso 7: Apriori y FP-Growth

In [1]:
# Importar librerías
import pandas as pd
import zipfile
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, apriori, association_rules

In [2]:
# Descomprimir el archivo ZIP y Cargar el archivo CSV
with zipfile.ZipFile('./Data/E-Commerce_evaluation.zip') as z:
    with z.open('E-Commerce_evaluation.csv') as f:
        data = pd.read_csv(f)

In [3]:
# Mostrar las primeras filas
data.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name
0,2425083,49125,1,2,18,NaN,17,1,0,13,pantry,baking ingredients
1,2425083,49125,1,2,18,NaN,91,2,0,16,dairy eggs,soy lactosefree
2,2425083,49125,1,2,18,NaN,36,3,0,16,dairy eggs,butter
3,2425083,49125,1,2,18,NaN,83,4,0,4,produce,fresh vegetables
4,2425083,49125,1,2,18,NaN,83,5,0,4,produce,fresh vegetables


El archivo contiene datos transaccionales con las siguientes columnas:

- order_id: Identifica cada pedido.
- user_id: Identifica al usuario que realiza el pedido.
- order_number: Número de pedido para cada usuario.
- order_dow y order_hour_of_day: Día de la semana y hora del pedido.
- days_since_prior_order: Días desde el último pedido.
- product_id y product_name: Identificadores y nombres de los productos.
- add_to_cart_order: Orden en que los productos fueron añadidos al carrito.
- reordered: Indica si el producto ya había sido ordenado antes.
- department y department_id: Departamento asociado al producto.

## Análisis exploratorio de datos

In [4]:
# Resumen del archivo
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2019501 entries, 0 to 2019500
Data columns (total 12 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   order_number            int64  
 3   order_dow               int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   product_id              int64  
 7   add_to_cart_order       int64  
 8   reordered               int64  
 9   department_id           int64  
 10  department              object 
 11  product_name            object 
dtypes: float64(1), int64(9), object(2)
memory usage: 184.9+ MB


Información general del conjunto de datos:

- Cantidad de registros: Hay 2,019,501 filas en el DataFrame.
- Cantidad de columnas: Existen 12 columnas que representan diferentes características relacionadas con cada transacción.
- Tamaño de memoria: El DataFrame ocupa 184.9 MB.
- Las columnas numéricas están correctamente categorizadas como int64 o float64.
- Las columnas de texto están en formato object.

Ahora, revisemos si hay valores nulos o duplicados para evitar problemas en el análisis posterior.

In [5]:
# Verificar valores nulos
print("Valores nulos por columna:")
data.isnull().sum()

Valores nulos por columna:


order_id                       0
user_id                        0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    124342
product_id                     0
add_to_cart_order              0
reordered                      0
department_id                  0
department                     0
product_name                   0
dtype: int64

La columna days_since_prior_order es la única que contiene valores nulos (124,342 valores).

In [6]:
# Verificar si hay columnas con valores idénticos
columnas_identicas = []

# Comparar cada columna con el resto
for i, col in enumerate(data.columns):
    for other_col in data.columns[i+1:]:
        if data[col].equals(data[other_col]):
            columnas_identicas.append((col, other_col))

# Mostrar columnas duplicadas, si existen
if columnas_identicas:
    print("Columnas con valores idénticos:")
    for dup in columnas_identicas:
        print(f"{dup[0]} es idéntica a {dup[1]}")
else:
    print("No hay columnas con los mismos valores en el DataFrame.")

No hay columnas con los mismos valores en el DataFrame.


In [7]:
# Verificar si hay filas duplicadas
duplicated_rows = data[data.duplicated()]

if not duplicated_rows.empty:
    print(f"Se encontraron {len(duplicated_rows)} filas duplicadas.")
    print("Primeras filas duplicadas encontradas:")
    print(duplicated_rows.head())
else:
    print("No se encontraron filas duplicadas en el DataFrame.")

No se encontraron filas duplicadas en el DataFrame.


In [8]:
# Número de productos únicos
print("Productos únicos:", data['product_name'].nunique())

Productos únicos: 134


In [9]:
# Transacciones únicas
print("Transacciones únicas:", data['order_id'].nunique())

Transacciones únicas: 200000


# Limpieza de datos

En este paso, se hará modificaciones en la columna days_since_prior_order para solucionar el problema de los valores nulos.

In [10]:
# Se sustituye los valores nulos con el valor 0
data['days_since_prior_order'] = data['days_since_prior_order'].fillna(0)

# Resumen de valores nulos
print("Valores nulos por columna:")
print(data.isnull().sum())

Valores nulos por columna:
order_id                  0
user_id                   0
order_number              0
order_dow                 0
order_hour_of_day         0
days_since_prior_order    0
product_id                0
add_to_cart_order         0
reordered                 0
department_id             0
department                0
product_name              0
dtype: int64


days_since_prior_order ahora no contiene valores nulos. Los valores nulos originales fueron reemplazados por 0, lo que puede interpretarse como usuarios que no tenían información sobre el pedido previo.

## Agrupar por order_id

El siguiente paso es agrupar los productos por order_id para formar las transacciones, lo que es esencial para los algoritmos de asociación.

In [11]:
# Agrupar productos por pedido (order_id) para formar transacciones
transactions = data.groupby('order_id')['product_name'].apply(list)

# Mostrar las primeras transacciones
transactions.head()

order_id
10    [fresh fruits, fresh vegetables, fresh herbs, ...
11    [frozen meals, fresh dips tapenades, canned me...
28    [butter, other creams cheeses, poultry counter...
38    [nuts seeds dried fruit, packaged vegetables f...
56    [canned jarred vegetables, packaged cheese, fr...
Name: product_name, dtype: object

Se agruparon los productos por el identificador del pedido (order_id), de manera que cada pedido ahora contiene una lista de los nombres de los productos comprados en ese pedido.

## Codificación One-Hot

La codificación One-Hot tiene como objetivo convertir las transacciones, que son listas de productos, en una matriz binaria que sea adecuada para los algoritmos de minería de datos como Apriori y FP-Growth. 

In [12]:
# Crear un DataFrame de transacciones con One-Hot Encoding
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

# Crear el DataFrame
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# Mostrar los primeros registros codificados
df_encoded.head()

,air fresheners candles,asian foods,baby accessories,baby bath body care,baby food formula,bakery desserts,baking ingredients,baking supplies decor,beauty,beers coolers,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
0,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


- Filas: Cada fila representa una transacción única, es decir, un pedido (order_id).
- Columnas: Cada columna corresponde a un producto único identificado por su nombre en product_name.
- Valores: Los valores son True o False, indicando si el producto está presente (True) o ausente (False) en esa transacción. Por ejemplo, en el registro tiene True en la columna "yogurt" y False en "white wines", significa que en esa transacción se compró yogurt pero no vino blanco.

## Filtrado por soporte mínimo

Antes de aplicar Apriori o Fpgrowth, primero filtramos los productos que cumplen con el soporte mínimo del 1% (0.01). Esto ayuda a reducir la carga computacional y memoria al trabajar solo con los productos más frecuentes.

In [13]:
# Contar las ocurrencias de cada producto
product_counts = df_encoded.sum(axis=0)

# Ajuste del soporte mínimo
min_support = 0.1 
min_count = min_support * len(df_encoded)

# Filtrar productos que no cumplen con el soporte mínimo
frequent_products = product_counts[product_counts >= min_count].index

# Filtrar el DataFrame solo con los productos frecuentes
df_filtered = df_encoded[frequent_products]

# Reducir el tamaño de los datos para pruebas, tomar solo el 10% de las transacciones
df_filtered_sampled = df_filtered.sample(frac=0.1, random_state=42)  

# Mostrar las primeras filas del DataFrame muestral
df_filtered_sampled.head()

,bread,chips pretzels,crackers,eggs,fresh fruits,fresh vegetables,frozen produce,ice cream ice,lunch meat,milk,packaged cheese,packaged vegetables fruits,refrigerated,soy lactosefree,water seltzer sparkling water,yogurt
119737,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False
72272,True,False,False,False,False,True,False,False,True,False,True,False,False,False,False,True
158154,True,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False
65426,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
30074,True,False,False,False,True,True,False,False,False,False,True,True,False,False,False,True


El análisis mostró que algunos productos son significativamente más populares que otros tales como Fresh Vegetables (Verduras Frescas), Bread (Pan) y Yogurt cumpliendo con el soporte mínimo del 10%. Estos productos frecuentes representan aquellos que los clientes tienden a comprar repetidamente en diferentes transacciones.

## Entrenar al modelo (Apriori)

Se aplicará el algoritmo Apriori para detectar conjuntos frecuentes de productos en el conjunto de datos de compras. Apriori nos ayuda a identificar combinaciones de productos que se compran con una frecuencia significativa, lo cual es fundamental para generar recomendaciones basadas en la compra conjunta de productos.

In [14]:
# Aplicar el algoritmo Apriori para obtener conjuntos frecuentes
frequent_itemsets_apriori = apriori(df_filtered_sampled, min_support=0.01, use_colnames=True)

# Mostrar los conjuntos frecuentes obtenidos
print("Conjuntos frecuentes (Apriori):")
print(frequent_itemsets_apriori)

# Calcular manualmente el número de columnas en 'frequent_itemsets'
num_itemsets = frequent_itemsets_apriori.shape[1]

# Generar las reglas de asociación basadas en la confianza
rules_apriori = association_rules(frequent_itemsets_apriori, num_itemsets=num_itemsets, metric="confidence", min_threshold=0.6)

# Mostrar las reglas de asociación
print("Reglas de asociación (Apriori):")
rules_apriori.head(7)

Conjuntos frecuentes (Apriori):
     support                                           itemsets
0    0.16275                                            (bread)
1    0.16575                                   (chips pretzels)
2    0.11625                                         (crackers)
3    0.13520                                             (eggs)
4    0.55990                                     (fresh fruits)
..       ...                                                ...
930  0.01035  (milk, bread, packaged cheese, packaged vegeta...
931  0.01140  (milk, yogurt, bread, packaged vegetables frui...
932  0.01200  (yogurt, bread, packaged cheese, packaged vege...
933  0.01025  (yogurt, packaged cheese, packaged vegetables ...
934  0.01625  (milk, yogurt, packaged cheese, packaged veget...

[935 rows x 2 columns]
Reglas de asociación (Apriori):


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(bread),(fresh fruits),0.16275,0.55990,0.11040,0.678341,1.211540,1.0,0.019276,1.368219,0.208545,0.180318,0.269123,0.437760
1,(chips pretzels),(fresh fruits),0.16575,0.55990,0.10285,0.620513,1.108257,1.0,0.010047,1.159723,0.117089,0.165141,0.137725,0.402103
2,(crackers),(fresh fruits),0.11625,0.55990,0.07425,0.638710,1.140757,1.0,0.009162,1.218134,0.139620,0.123359,0.179072,0.385661
3,(eggs),(fresh fruits),0.13520,0.55990,0.09730,0.719675,1.285363,1.0,0.021602,1.569961,0.256718,0.162763,0.363041,0.446728
4,(eggs),(fresh vegetables),0.13520,0.44685,0.08270,0.611686,1.368885,1.0,0.022286,1.424493,0.311608,0.165615,0.297996,0.398380
5,(fresh vegetables),(fresh fruits),0.44685,0.55990,0.32165,0.719816,1.285616,1.0,0.071459,1.570756,0.401632,0.469494,0.363364,0.647147
6,(frozen produce),(fresh fruits),0.12270,0.55990,0.09005,0.733904,1.310777,1.0,0.021350,1.653913,0.270254,0.151970,0.395373,0.447368


- Conjuntos Frecuentes: El algoritmo Apriori identificó productos que se compran frecuentemente juntos.
- Reglas de Asociación: Las reglas muestran que si un usuario compra un producto (como "pan"), es probable que también compre otro (como "frutas frescas"), con una confianza del 67.83%.
- Métricas: El lift mayor a 1 indica que los productos están asociados más de lo que se esperaría por azar, lo que sugiere asociaciones útiles para recomendaciones.

In [15]:
# Filtrar reglas fuertes con confianza > 0.6 y lift > 1.2
strong_rules_apriori = rules_apriori[(rules_apriori['confidence'] > 0.6) & (rules_apriori['lift'] > 1.2)]

# Mostrar las reglas fuertes
print("Reglas fuertes de asociación:")
strong_rules_apriori.head(7)

Reglas fuertes de asociación:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(bread),(fresh fruits),0.16275,0.55990,0.11040,0.678341,1.211540,1.0,0.019276,1.368219,0.208545,0.180318,0.269123,0.437760
3,(eggs),(fresh fruits),0.13520,0.55990,0.09730,0.719675,1.285363,1.0,0.021602,1.569961,0.256718,0.162763,0.363041,0.446728
4,(eggs),(fresh vegetables),0.13520,0.44685,0.08270,0.611686,1.368885,1.0,0.022286,1.424493,0.311608,0.165615,0.297996,0.398380
5,(fresh vegetables),(fresh fruits),0.44685,0.55990,0.32165,0.719816,1.285616,1.0,0.071459,1.570756,0.401632,0.469494,0.363364,0.647147
6,(frozen produce),(fresh fruits),0.12270,0.55990,0.09005,0.733904,1.310777,1.0,0.021350,1.653913,0.270254,0.151970,0.395373,0.447368
7,(lunch meat),(fresh fruits),0.10605,0.55990,0.07425,0.700141,1.250476,1.0,0.014873,1.467692,0.224067,0.125486,0.318658,0.416377
9,(packaged cheese),(fresh fruits),0.22835,0.55990,0.15435,0.675936,1.207244,1.0,0.026497,1.358065,0.222468,0.243493,0.263658,0.475805


- Las reglas fuertes muestran productos que tienen una alta probabilidad de ser comprados juntos, como "pan" y "frutas frescas" con una confianza del 67.83%.
- Lift > 1.2 indica que la relación entre los productos es significativamente mayor que la que ocurriría por azar.

## Entrenar al modelo (FP-Growth)

Se aplicará el algoritmo FP-Growth para encontrar conjuntos frecuentes de productos comprados juntos. Este algoritmo es similar a Apriori, pero es más eficiente en cuanto a tiempo de ejecución, especialmente en grandes bases de datos.

In [16]:
# Aplicar FP-Growth para obtener los conjuntos frecuentes
frequent_itemsets_fpgrowth = fpgrowth(df_filtered_sampled, min_support=0.01, use_colnames=True)

# Mostrar los conjuntos frecuentes obtenidos
print("Conjuntos frecuentes (FP-Growth):")
print(frequent_itemsets_fpgrowth)

# Calcular el número de itemsets (columnas) en 'frequent_itemsets'
num_itemsets_fpgrowth = frequent_itemsets_fpgrowth.shape[1]

# Calcular las reglas de asociación basadas en las frecuencias
rules_fpgrowth = association_rules(frequent_itemsets_fpgrowth, num_itemsets=num_itemsets_fpgrowth, metric="confidence", min_threshold=0.6)

# Mostrar las reglas de asociación generadas
print("Reglas de asociación (FP-Growth):")
rules_fpgrowth.head()

Conjuntos frecuentes (FP-Growth):
     support                                           itemsets
0    0.19025                    (water seltzer sparkling water)
1    0.11625                                         (crackers)
2    0.44685                                 (fresh vegetables)
3    0.26950                                           (yogurt)
4    0.22835                                  (packaged cheese)
..       ...                                                ...
930  0.01310  (fresh fruits, packaged cheese, packaged veget...
931  0.01220  (ice cream ice, packaged cheese, packaged vege...
932  0.01005  (ice cream ice, packaged cheese, packaged vege...
933  0.01145  (ice cream ice, packaged cheese, fresh fruits,...
934  0.01560  (ice cream ice, packaged cheese, fresh fruits,...

[935 rows x 2 columns]
Reglas de asociación (FP-Growth):


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,"(water seltzer sparkling water, fresh vegetables)",(fresh fruits),0.08420,0.55990,0.06450,0.766033,1.368161,1.0,0.017356,1.881037,0.293832,0.111284,0.468378,0.440616
1,"(water seltzer sparkling water, packaged veget...",(fresh fruits),0.07480,0.55990,0.05875,0.785428,1.402800,1.0,0.016869,2.051058,0.310355,0.102005,0.512447,0.445179
2,"(water seltzer sparkling water, packaged veget...",(fresh vegetables),0.07480,0.44685,0.04715,0.630348,1.410647,1.0,0.013726,1.496406,0.314641,0.099368,0.331732,0.367932
3,"(water seltzer sparkling water, packaged veget...",(fresh vegetables),0.05875,0.44685,0.03915,0.666383,1.491290,1.0,0.012898,1.658039,0.350002,0.083932,0.396878,0.376998
4,"(water seltzer sparkling water, fresh fruits, ...",(packaged vegetables fruits),0.06450,0.36380,0.03915,0.606977,1.668435,1.0,0.015685,1.618734,0.428259,0.100604,0.382233,0.357295


- Conjuntos Frecuentes: El algoritmo FP-Growth identificó productos que se compran frecuentemente juntos, como "agua con gas y vegetales frescos".
- Reglas de Asociación: Las reglas indican que si un usuario compra ciertos productos, como "agua con gas y vegetales frescos", es probable que también compre otros, como "frutas frescas", con una alta confianza (76.6%).
- Métricas: El lift mayor a 1 muestra que las asociaciones entre productos son más fuertes de lo que se esperaría por azar, lo que sugiere patrones útiles para recomendaciones.

In [17]:
# Filtrar reglas fuertes con confianza > 0.6 y lift > 1.2
strong_rules_fpgrowth = rules_fpgrowth[(rules_fpgrowth['confidence'] > 0.6) & (rules_fpgrowth['lift'] > 1.2)]

# Mostrar las reglas fuertes
print("Reglas fuertes de asociación (FP-Growth):")
strong_rules_fpgrowth.head()

Reglas fuertes de asociación (FP-Growth):


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,"(water seltzer sparkling water, fresh vegetables)",(fresh fruits),0.08420,0.55990,0.06450,0.766033,1.368161,1.0,0.017356,1.881037,0.293832,0.111284,0.468378,0.440616
1,"(water seltzer sparkling water, packaged veget...",(fresh fruits),0.07480,0.55990,0.05875,0.785428,1.402800,1.0,0.016869,2.051058,0.310355,0.102005,0.512447,0.445179
2,"(water seltzer sparkling water, packaged veget...",(fresh vegetables),0.07480,0.44685,0.04715,0.630348,1.410647,1.0,0.013726,1.496406,0.314641,0.099368,0.331732,0.367932
3,"(water seltzer sparkling water, packaged veget...",(fresh vegetables),0.05875,0.44685,0.03915,0.666383,1.491290,1.0,0.012898,1.658039,0.350002,0.083932,0.396878,0.376998
4,"(water seltzer sparkling water, fresh fruits, ...",(packaged vegetables fruits),0.06450,0.36380,0.03915,0.606977,1.668435,1.0,0.015685,1.618734,0.428259,0.100604,0.382233,0.357295


- Las reglas fuertes muestran productos que tienen una alta probabilidad de ser comprados juntos, como "agua con gas y vegetales frescos" con una confianza del 76.6%.
- Un lift > 1.3 indica que la relación entre los productos es significativamente mayor que la que ocurriría por azar.

## Predecir los productos que un usuario podría comprar en su próximo pedido

Se implementará una función para predecir los productos que un usuario podría comprar en su próximo pedido, basándose en las compras previas del usuario y las reglas de asociación generadas. Esta función sugiere productos relacionados con las compras anteriores del usuario, ayudando a mejorar las recomendaciones.

In [18]:
def predict_next_purchase(user_purchased_products, rules, top_n=5):
    recommendations = []
    
    # Iterar sobre las reglas
    for _, row in rules.iterrows():
        # Verificar si el producto antecedente ya fue comprado por el usuario
        antecedents = list(row['antecedents'])
        
        if any(product in user_purchased_products for product in antecedents):
            # Si el antecedente fue comprado, agregar el consecuente a las recomendaciones
            consequents = list(row['consequents'])
            recommendations.extend(consequents)
    
    # Eliminar duplicados de las recomendaciones y ordenar por la frecuencia de las reglas
    recommendations = list(set(recommendations))
    
    # Seleccionar las 'top_n' recomendaciones
    return recommendations[:top_n]

## Ejecución del modelo Apriori

Se ejecutará el modelo Apriori para predecir los productos que un usuario podría comprar en su próximo pedido. Se toma una lista de productos que el usuario ya ha comprado y se recomienda una lista de productos que tienen una alta probabilidad de ser comprados a continuación, utilizando las reglas de asociación que muestran qué productos suelen comprarse juntos.

In [19]:
user_purchased_products = ['bread', 'milk', 'eggs']  # Productos que el usuario ya ha comprado
recommended_products = predict_next_purchase(user_purchased_products, strong_rules_apriori, top_n=5)

print("Productos recomendados para el próximo pedido (Apriori):")
print(recommended_products)

Productos recomendados para el próximo pedido (Apriori):
['yogurt', 'packaged cheese', 'packaged vegetables fruits', 'fresh vegetables', 'fresh fruits']


Dado que el usuario ya ha comprado productos como 'bread', 'milk' y 'eggs', el modelo recomienda productos que suelen ser comprados junto con estos, como 'fresh fruits', 'yogurt' y 'fresh vegetables'. Estas recomendaciones están basadas en la probabilidad de que, al comprar ciertos productos, otros también sean comprados.

## Ejecución del modelo FP-Growth

Se ejecutará el modelo FP-Growth para predecir los productos que un usuario podría comprar en su próximo pedido. Al igual que con el modelo Apriori, se utiliza una lista de productos que el usuario ya ha comprado para recomendar productos que tienen una alta probabilidad de ser comprados juntos.

In [20]:
user_purchased_products = ['bread', 'milk', 'eggs']  # Productos que el usuario ya ha comprado
recommended_products_fpgrowth = predict_next_purchase(user_purchased_products, strong_rules_fpgrowth, top_n=5)

print("Productos recomendados para el próximo pedido (FP-Growth):")
print(recommended_products_fpgrowth)

Productos recomendados para el próximo pedido (FP-Growth):
['yogurt', 'packaged cheese', 'packaged vegetables fruits', 'fresh vegetables', 'fresh fruits']


En este caso, las recomendaciones obtenidas son idénticas a las generadas por el modelo Apriori, como 'fresh fruits', 'yogurt' y 'fresh vegetables', lo que sugiere que ambos modelos están funcionando de manera similar en este escenario específico.